In [ ]:
import torch
# The train split of client 1
cl_num = 0
train_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/train.pt')
val_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/val.pt')
test_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/test.pt')
print(len(train_data_client) + len(val_data_client) + len(test_data_client))


1: ESOL(12)
2: FreeSolv(11)
3: Lipophilicity (13)
4: BACE (7)
5: BBBP (8)
6: ClinTox (6)
7: MUTAG (1)
8: PTC_MR (3)
9: PTC_MM (2)
10:  PTC_FM (4)
11: PTC_FR (5) Zu viele Datem: 1440
12: NCI109 (10)
13: NCI1 (9)
14: alchemy_full (15)
15: ZINC_full (16)
16: QM9 (14) (zu viele Daten? 130831)

In [ ]:
len(train_data_client)

In [1]:
import os
import sys

# from federatedscope.core.cmd_args import parse_args
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.auxiliaries.worker_builder import get_client_cls, get_server_cls
from federatedscope.core.configs.config import global_cfg
from federatedscope.core.fed_runner import FedRunner
from yacs.config import CfgNode
from rdkit.Chem.Draw import IPythonConsole

if os.environ.get('https_proxy'):
    del os.environ['https_proxy']
if os.environ.get('http_proxy'):
    del os.environ['http_proxy']

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
2022-12-22 19:23:48,197 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [2]:
def train(client, lr):
    cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/isolated.yaml'
    cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_isolated.yaml'
    #'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

    init_cfg = global_cfg.clone()
    init_cfg.merge_from_file(cfg_file)

    # init_cfg.data.subdirectory = 'graph_dt_backup/processed'
    # init_cfg.merge_from_list(args.opts)
    init_cfg.data.client = client
    init_cfg.train.optimizer.lr = lr
    update_logger(init_cfg)
    setup_seed(init_cfg.seed)

    # federated dataset might change the number of clients
    # thus, we allow the creation procedure of dataset to modify the global cfg object
    data, modified_cfg = get_data(config=init_cfg.clone())
    init_cfg.merge_from_other_cfg(modified_cfg)

    init_cfg.freeze()

    # allow different settings for different clients
    # cfg_client.merge_from_file(args.cfg_client)
    if cfg_client is None:
        cfg_client = None
    else:
        cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()
    runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
    _ = runner.run()

In [ ]:
clients = range(1, 16+1)
lrs = [0.1, 0.0001]
num_trainings = 3
for client in clients:
    for lr in lrs:
        print(f"Client: {client},\tlr: {lr}")
        for i in range(num_trainings):
            print(f"training run: {i+1}")
            train(client, lr)

2022-12-22 19:23:52,640 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:23:52,642 (utils:130)INFO: the current machine is at 127.0.1.1
2022-12-22 19:23:52,644 (utils:132)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-12-22 19:23:52,644 (utils:133)INFO: the output dir is exp/CLIENT_1/local_gin_on_graph-dt_lr0.1_lstep1_


Client: 1,	lr: 0.1
training run: 1


2022-12-22 19:23:54,113 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:23:54,114 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:23:54,129 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  client: 1
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-22 19:23:56,353 (client:260)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_loss': 4812.948519, 'train_avg_loss': 5.341785, 'train_total': 901}}
2022-12-22 19:23:56,355 (server:323)INFO: Server #0: Starting evaluation at the end of round 0.
2022-12-22 19:23:56,356 (server:330)INFO: ----------- Starting a new training round (Round #1) -------------
2022-12-22 19:23:56,389 (client:415)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'test_loss': 778.27937, 'test_avg_loss': 6.887428, 'test_total': 113, 'val_loss': 638.523252, 'val_avg_loss': 5.650648, 'val_total': 113}}
2022-12-22 19:23:56,391 (monitor:936)INFO: Current best: {'test_loss': 778.27937, 'test_avg_loss': 6.887428, 'test_total': 113, 'val_loss': 638.523252, 'val_avg_loss': 5.650648, 'val_total': 113}
2022-12-22 19:23:56,535 (client:260)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'train_loss': 3552.589839, 'train_avg_loss': 3.942941, 'train_total': 901}}
2022-12-22 19:23:56,537 (server:

In [2]:
cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/isolated.yaml'
cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_isolated.yaml'
#'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

init_cfg = global_cfg.clone()
init_cfg.merge_from_file(cfg_file)

# init_cfg.data.subdirectory = 'graph_dt_backup/processed'
# init_cfg.merge_from_list(args.opts)
init_cfg.data.client = 5
init_cfg.train.optimizer.lr = 0.01
update_logger(init_cfg)
setup_seed(init_cfg.seed)

# federated dataset might change the number of clients
# thus, we allow the creation procedure of dataset to modify the global cfg object
data, modified_cfg = get_data(config=init_cfg.clone())
init_cfg.merge_from_other_cfg(modified_cfg)

init_cfg.freeze()

# allow different settings for different clients
# cfg_client.merge_from_file(args.cfg_client)
if cfg_client is None:
    cfg_client = None
else:
    cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()



2022-12-22 19:14:26,477 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:14:26,480 (utils:130)INFO: the current machine is at 127.0.1.1
2022-12-22 19:14:26,480 (utils:132)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-12-22 19:14:26,481 (utils:133)INFO: the output dir is exp/CLIENT_5/local_gin_on_graph-dt_lr0.01_lstep1_/sub_exp_20221222191426
2022-12-22 19:14:30,187 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:14:30,188 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:14:30,210 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  cla

In [3]:
runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
_ = runner.run()

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2022-12-22 19:14:30,329 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-12-22 19:14:30,331 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:14:30,332 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-22 19:14:30,349 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1


track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-22 19:14:32,644 (client:260)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_acc': 0.613121, 'train_avg_loss': 0.673806, 'train_loss': 1098.977693, 'train_total': 1631}}
2022-12-22 19:14:32,646 (server:323)INFO: Server #0: Starting evaluation at the end of round 0.
2022-12-22 19:14:32,646 (server:330)INFO: ----------- Starting a new training round (Round #1) -------------
2022-12-22 19:14:32,702 (client:415)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'test_acc': 0.745098, 'test_avg_loss': 0.653336, 'test_loss': 133.280626, 'test_total': 204, 'val_acc': 0.77451, 'val_avg_loss': 0.648659, 'val_loss': 132.326364, 'val_total': 204}}
2022-12-22 19:14:32,703 (monitor:936)INFO: Current best: {'test_acc': 0.745098, 'test_avg_loss': 0.653336, 'test_loss': 133.280626, 'test_total': 204, 'val_acc': 0.77451, 'val_avg_loss': 0.648659, 'val_loss': 132.326364, 'val_total': 204}


no improve counter: 0
self.best_metric: 0.6486586460880205


2022-12-22 19:14:32,957 (client:260)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'train_acc': 0.754752, 'train_avg_loss': 0.625892, 'train_loss': 1020.829178, 'train_total': 1631}}
2022-12-22 19:14:32,958 (server:496)INFO: {'Role': 'Server #', 'Round': 1, 'Results_avg': {'test_acc': 0.745098, 'test_avg_loss': 0.653336, 'test_loss': 133.280626, 'test_total': 204.0, 'val_acc': 0.77451, 'val_avg_loss': 0.648659, 'val_loss': 132.326364, 'val_total': 204.0}}
2022-12-22 19:14:32,960 (monitor:936)INFO: Current best: {'test_acc': [0.745098], 'test_avg_loss': [0.653336], 'test_loss': [133.280626], 'test_total': [204.0], 'val_acc': [0.77451], 'val_avg_loss': [0.648659], 'val_loss': [132.326364], 'val_total': [204.0]}
2022-12-22 19:14:32,963 (monitor:936)INFO: Current best: {'test_acc': 0.745098, 'test_avg_loss': 0.653336, 'test_loss': 133.280626, 'test_total': 204.0, 'val_acc': 0.77451, 'val_avg_loss': 0.648659, 'val_loss': 132.326364, 'val_total': 204.0}
2022-12-22 19:14:32,967 (serv

...

no improve counter: 47
self.best_metric: 72.49091899394989
no improve counter: 48
self.best_metric: 0.3553476421272053


2022-12-22 19:16:00,217 (client:260)INFO: {'Role': 'Client #1', 'Round': 258, 'Results_raw': {'train_acc': 0.882281, 'train_avg_loss': 0.32236, 'train_loss': 525.769836, 'train_total': 1631}}
2022-12-22 19:16:00,219 (server:496)INFO: {'Role': 'Server #', 'Round': 258, 'Results_avg': {'test_acc': 0.808824, 'test_avg_loss': 0.487379, 'test_loss': 99.425253, 'test_total': 204.0, 'val_acc': 0.848039, 'val_avg_loss': 0.38871, 'val_loss': 79.296792, 'val_total': 204.0}}
2022-12-22 19:16:00,220 (monitor:936)INFO: Current best: {'test_acc': 0.813725, 'test_avg_loss': 0.47469, 'test_loss': 96.836845, 'test_total': 204.0, 'val_acc': 0.857843, 'val_avg_loss': 0.355348, 'val_loss': 72.490919, 'val_total': 204.0}
2022-12-22 19:16:00,222 (monitor:936)INFO: Current best: {'test_acc': 0.813725, 'test_avg_loss': 0.47469, 'test_loss': 96.836845, 'test_total': 204.0, 'val_acc': 0.857843, 'val_avg_loss': 0.355348, 'val_loss': 72.490919, 'val_total': 204.0}
2022-12-22 19:16:00,224 (server:323)INFO: Server 

no improve counter: 48
self.best_metric: 72.49091899394989
no improve counter: 49
self.best_metric: 0.3553476421272053


2022-12-22 19:16:00,550 (client:260)INFO: {'Role': 'Client #1', 'Round': 259, 'Results_raw': {'train_acc': 0.868179, 'train_avg_loss': 0.32694, 'train_loss': 533.239336, 'train_total': 1631}}
2022-12-22 19:16:00,553 (server:496)INFO: {'Role': 'Server #', 'Round': 259, 'Results_avg': {'test_acc': 0.813725, 'test_avg_loss': 0.450019, 'test_loss': 91.803892, 'test_total': 204.0, 'val_acc': 0.843137, 'val_avg_loss': 0.365372, 'val_loss': 74.535888, 'val_total': 204.0}}
2022-12-22 19:16:00,554 (monitor:936)INFO: Current best: {'test_acc': 0.813725, 'test_avg_loss': 0.47469, 'test_loss': 96.836845, 'test_total': 204.0, 'val_acc': 0.857843, 'val_avg_loss': 0.355348, 'val_loss': 72.490919, 'val_total': 204.0}
2022-12-22 19:16:00,556 (monitor:936)INFO: Current best: {'test_acc': 0.813725, 'test_avg_loss': 0.47469, 'test_loss': 96.836845, 'test_total': 204.0, 'val_acc': 0.857843, 'val_avg_loss': 0.355348, 'val_loss': 72.490919, 'val_total': 204.0}
2022-12-22 19:16:00,558 (server:323)INFO: Server

no improve counter: 49
self.best_metric: 72.49091899394989
no improve counter: 50
self.best_metric: 0.3553476421272053
no improve counter: 50
self.best_metric: 72.49091899394989


In [4]:
1

1

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


In [ ]:
cfg_client['client_5']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_6']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_7']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_8']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_9']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_10']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_11']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_12']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_13']['eval']['metrics']=['mse']

In [ ]:
cl_list = ['client_' + str(i) for i in range(1,17)]

In [ ]:
cl_list

In [ ]:
cfg_client['client_13']['train']['local_update_steps']

In [ ]:
for el in cl_list:
    cfg_client[el]['train']['local_update_steps']=1

In [ ]:
cfg_client